# 时序神经网络综述 参考资料：https://blog.csdn.net/zy_dreamer/article/details/136211280

# 使用Transformer搭建时空神经网络。但是单独使用transformer对时间的预测结果不是很友好，如想使用，可以使用transformer与其他模型相结合的方式。 参考资料：https://blog.csdn.net/java1314777/article/details/134355884。 如想理解transformer请详见：https://blog.csdn.net/m0_37605642/article/details/135958384

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskTransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, seq_len, capacity_len):
        super(MultiTaskTransformerModel, self).__init__()
        
        # Positional Encoding
        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_len, d_model))
        
        # Input Linear Transformation
        self.input_layer = nn.Linear(input_dim, d_model)
        
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        
        # Output Branches
        # Hourly generation resource prediction (seq_len)
        self.hourly_branch = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Linear(d_model // 2, 1)
        )
        
        # Capacity prediction (capacity_len)
        self.capacity_branch = nn.Sequential(
            nn.Linear(seq_len * d_model, d_model),
            nn.ReLU(),
            nn.Linear(d_model, capacity_len)
        )

    def forward(self, x):
        # Add positional encoding
        x = self.input_layer(x) + self.positional_encoding
        
        # Transformer Encoder
        x = self.transformer_encoder(x)
        
        # Hourly generation resource prediction
        hourly_output = self.hourly_branch(x).squeeze(-1)
        
        # Capacity prediction
        flat_features = x.flatten(start_dim=1)
        capacity_output = self.capacity_branch(flat_features)

        return hourly_output, capacity_output

# Model parameters
input_dim = 2           # Two input features: wind speed u and v
d_model = 64            # Transformer embedding dimension
nhead = 4               # Number of attention heads
num_encoder_layers = 3  # Number of transformer layers
dim_feedforward = 128   # Dimension of the feedforward layer
dropout = 0.1           # Dropout rate
seq_len = 8760          # Length of the time series (hours in a year)
capacity_len = 176      # Length of the wind power capacity output

# Initialize the model
model = MultiTaskTransformerModel(
    input_dim, d_model, nhead, num_encoder_layers, dim_feedforward, dropout, seq_len, capacity_len
)

# Example inputs
batch_size = 32
inputs = torch.rand(batch_size, seq_len, input_dim)  # Random input tensor

# Forward pass
hourly_output, capacity_output = model(inputs)

print(f"Hourly Output Shape: {hourly_output.shape}")  # Expected: (batch_size, seq_len)
print(f"Capacity Output Shape: {capacity_output.shape}")  # Expected: (batch_size, capacity_len)

# Define loss functions
criterion_hourly = nn.MSELoss()
criterion_capacity = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Example target outputs
hourly_target = torch.rand(batch_size, seq_len)
capacity_target = torch.rand(batch_size, capacity_len)

# Training step
optimizer.zero_grad()
hourly_pred, capacity_pred = model(inputs)

loss_hourly = criterion_hourly(hourly_pred, hourly_target)
loss_capacity = criterion_capacity(capacity_pred, capacity_target)

# Combined loss
loss = loss_hourly + loss_capacity
loss.backward()
optimizer.step()

print(f"Training Loss: {loss.item()}")


E:\Anaconda\AnacondaInstall\envs\pytorch\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Hourly Output Shape: torch.Size([32, 8760])
Capacity Output Shape: torch.Size([32, 176])


KeyboardInterrupt: 

# 卷积是为了跨时间步骤提取特征，我们这里可能不需要进行跨时间步骤提取特征的需求，故舍弃。参考资料为：https://blog.csdn.net/Leon_winter/article/details/100124146 使用TCN处理时间序列

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class TemporalConvNet(nn.Module):
    def __init__(self, input_dim, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            in_channels = input_dim if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [
                nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=(kernel_size-1), dilation=1),
                nn.ReLU(),
                nn.Dropout(dropout)
            ]
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class MultiTaskTCNModel(nn.Module):
    def __init__(self, input_dim, num_channels, seq_len, capacity_len, kernel_size=2, dropout=0.2):
        super(MultiTaskTCNModel, self).__init__()
        self.tcn = TemporalConvNet(input_dim, num_channels, kernel_size, dropout)
        self.hourly_branch = nn.Sequential(
            nn.Linear(num_channels[-1], num_channels[-1] // 2),
            nn.ReLU(),
            nn.Linear(num_channels[-1] // 2, 1)
        )
        self.capacity_branch = nn.Sequential(
            nn.Linear(seq_len * num_channels[-1], num_channels[-1]),
            nn.ReLU(),
            nn.Linear(num_channels[-1], capacity_len)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Reshape for Conv1d (batch, input_dim, seq_len)
        features = self.tcn(x).permute(0, 2, 1)  # Reshape back (batch, seq_len, features)
        hourly_output = self.hourly_branch(features).squeeze(-1)  # (batch, seq_len)
        flat_features = features.flatten(start_dim=1)  # Flatten for capacity branch
        capacity_output = self.capacity_branch(flat_features)  # (batch, capacity_len)
        return hourly_output, capacity_output

# Model parameters
input_dim = 2           # Two input features: wind speed u and v
num_channels = [16, 32, 64]  # TCN channel sizes for each layer
seq_len = 8760          # Length of the time series (hours in a year)
capacity_len = 176      # Length of the wind power capacity output
kernel_size = 3         # Kernel size for TCN
dropout = 0.1           # Dropout rate

# Initialize the model
model = MultiTaskTCNModel(input_dim, num_channels, seq_len, capacity_len, kernel_size, dropout)

# Example inputs
batch_size = 32
inputs = torch.rand(batch_size, seq_len, input_dim)  # Random input tensor

# Forward pass
hourly_output, capacity_output = model(inputs)

print(f"Hourly Output Shape: {hourly_output.shape}")  # Expected: (batch_size, seq_len)
print(f"Capacity Output Shape: {capacity_output.shape}")  # Expected: (batch_size, capacity_len)

# Define loss functions
criterion_hourly = nn.MSELoss()
criterion_capacity = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Example target outputs
hourly_target = torch.rand(batch_size, seq_len)
capacity_target = torch.rand(batch_size, capacity_len)

# Training step
optimizer.zero_grad()
hourly_pred, capacity_pred = model(inputs)

loss_hourly = criterion_hourly(hourly_pred, hourly_target)
loss_capacity = criterion_capacity(capacity_pred, capacity_target)

# Combined loss
loss = loss_hourly + loss_capacity
loss.backward()
optimizer.step()

print(f"Training Loss: {loss.item()}")


# （可以使用，既可以解决梯度消失和梯度爆炸问题，还能够训练较好的效果。参考资料：https://blog.csdn.net/mary19831/article/details/129570030 ）使用LSTM处理时间序列神经网络

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, seq_len, capacity_len, dropout):
        super(MultiTaskLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        # Hourly generation resource prediction branch
        self.hourly_branch = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )

        # Capacity prediction branch
        self.capacity_branch = nn.Sequential(
            nn.Linear(seq_len * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, capacity_len)
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # print(type(lstm_out))
        # print(lstm_out.shape)
        # LSTM output: (batch, seq_len, hidden_dim)
        hourly_output = self.hourly_branch(lstm_out).squeeze(-1)  # (batch, seq_len)

        flat_features = lstm_out.flatten(start_dim=1)  # Flatten for capacity branch
        capacity_output = self.capacity_branch(flat_features)  # (batch, capacity_len)

        return hourly_output, capacity_output



In [3]:
modeltest = MultiTaskLSTMModel(2,64,2,8760,176,0.1)
print(modeltest)

MultiTaskLSTMModel(
  (lstm): LSTM(2, 64, num_layers=2, batch_first=True, dropout=0.1)
  (hourly_branch): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=1, bias=True)
  )
  (capacity_branch): Sequential(
    (0): Linear(in_features=560640, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=176, bias=True)
  )
)


In [2]:
# Model parameters
input_dim = 2           # Two input features: wind speed u and v
hidden_dim = 64         # Hidden dimension for LSTM
num_layers = 2          # Number of LSTM layers
seq_len = 8760          # Length of the time series (hours in a year)
capacity_len = 176      # Length of the wind power capacity output
dropout = 0.1           # Dropout rate

# Initialize the model
model = MultiTaskLSTMModel(input_dim, hidden_dim, num_layers, seq_len, capacity_len, dropout)

# Example inputs
batch_size = 32
inputs = torch.rand(batch_size, seq_len, input_dim)  # Random input tensor
print(inputs.shape)
# Forward pass
hourly_output, capacity_output = model(inputs)

print(f"Hourly Output Shape: {hourly_output.shape}")  # Expected: (batch_size, seq_len)
print(f"Capacity Output Shape: {capacity_output.shape}")  # Expected: (batch_size, capacity_len)

# Define loss functions
criterion_hourly = nn.MSELoss()
criterion_capacity = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Example target outputs
hourly_target = torch.rand(batch_size, seq_len)
capacity_target = torch.rand(batch_size, capacity_len)

# Training step
optimizer.zero_grad()
hourly_pred, capacity_pred = model(inputs)

loss_hourly = criterion_hourly(hourly_pred, hourly_target)
loss_capacity = criterion_capacity(capacity_pred, capacity_target)

# Combined loss
loss = loss_hourly + loss_capacity
loss.backward()
optimizer.step()

print(f"Training Loss: {loss.item()}")


torch.Size([32, 8760, 2])
Hourly Output Shape: torch.Size([32, 8760])
Capacity Output Shape: torch.Size([32, 176])
Training Loss: 0.8657733201980591


In [ ]:
import torch
import torch.nn as nn

class SpatiotemporalLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_len, capacity_len, dropout=0.1):
        super(SpatiotemporalLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=1, batch_first=True, dropout=dropout)

        # Hourly generation resource prediction branch
        self.hourly_branch = nn.Linear(hidden_dim, 1)  # Directly map hidden state to output

        # Capacity prediction branch
        self.capacity_branch = nn.Sequential(
            nn.Linear(seq_len * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, capacity_len)
        )

    def forward(self, x):
        # LSTM layer
        lstm_out, _ = self.lstm(x)  # lstm_out shape: (batch_size, seq_len, hidden_dim)

        # Hourly prediction: apply linear layer to each time step
        hourly_output = self.hourly_branch(lstm_out).squeeze(-1)  # Shape: (batch_size, seq_len)

        # Capacity prediction: flatten LSTM outputs and pass through MLP
        flat_features = lstm_out.flatten(start_dim=1)  # Shape: (batch_size, seq_len * hidden_dim)
        capacity_output = self.capacity_branch(flat_features)  # Shape: (batch_size, capacity_len)

        return hourly_output, capacity_output


# （由于GRU比LSTM简单易训，但是表达能力可能没有LSTM好，故舍弃。参考资料：https://blog.csdn.net/Michale_L/article/details/122778270 ）使用GRU进行时间序列神经网络搭建

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskGRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, seq_len, capacity_len, dropout):
        super(MultiTaskGRUModel, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)

        # Hourly generation resource prediction branch
        self.hourly_branch = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1)
        )

        # Capacity prediction branch
        self.capacity_branch = nn.Sequential(
            nn.Linear(seq_len * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, capacity_len)
        )

    def forward(self, x):
        gru_out, _ = self.gru(x)  # GRU output: (batch, seq_len, hidden_dim)
        hourly_output = self.hourly_branch(gru_out).squeeze(-1)  # (batch, seq_len)

        flat_features = gru_out.flatten(start_dim=1)  # Flatten for capacity branch
        capacity_output = self.capacity_branch(flat_features)  # (batch, capacity_len)

        return hourly_output, capacity_output

# Model parameters
input_dim = 2           # Two input features: wind speed u and v
hidden_dim = 64         # Hidden dimension for GRU
num_layers = 2          # Number of GRU layers
seq_len = 8760          # Length of the time series (hours in a year)
capacity_len = 176      # Length of the wind power capacity output
dropout = 0.1           # Dropout rate

# Initialize the model
model = MultiTaskGRUModel(input_dim, hidden_dim, num_layers, seq_len, capacity_len, dropout)

# Example inputs
batch_size = 32
inputs = torch.rand(batch_size, seq_len, input_dim)  # Random input tensor

# Forward pass
hourly_output, capacity_output = model(inputs)

print(f"Hourly Output Shape: {hourly_output.shape}")  # Expected: (batch_size, seq_len)
print(f"Capacity Output Shape: {capacity_output.shape}")  # Expected: (batch_size, capacity_len)

# Define loss functions
criterion_hourly = nn.MSELoss()
criterion_capacity = nn.MSELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Example target outputs
hourly_target = torch.rand(batch_size, seq_len)
capacity_target = torch.rand(batch_size, capacity_len)

# Training step
optimizer.zero_grad()
hourly_pred, capacity_pred = model(inputs)

loss_hourly = criterion_hourly(hourly_pred, hourly_target)
loss_capacity = criterion_capacity(capacity_pred, capacity_target)

# Combined loss
loss = loss_hourly + loss_capacity
loss.backward()
optimizer.step()

print(f"Training Loss: {loss.item()}")


# 一般的RNN由于不能解决梯度消失和梯度爆炸的问题，故一般不适用RNN。参考资料：https://blog.csdn.net/qq_51320133/article/details/138213246

# 图神经网络。可以把我们需要的时间和离散空间的点传入进去，不过可能导致维度过大，处理时间长，且用不上我们提取到的特征。好处是，可以训练出可能存在的空间的关系，以帮助模型找到更好的效果。参考资料：https://blog.csdn.net/deephub/article/details/137769276

# 个人觉得使用LSTM和GRU组合训练形成多元预测是最佳路径。 参考资料：https://blog.csdn.net/java1314777/article/details/134272675